In [0]:
select 
distinct product_name
, productgroup_name , lakehouse_updated_date
from pricing_analytics.silver.daily_pricing_silver
where product_name= 'Onion'

In [0]:
update pricing_analytics.silver.daily_pricing_silver
set productgroup_name = "Vegetables",
lakehouse_updated_date = current_timestamp()
where product_name= "Onion"

In [0]:
create table if not exists pricing_analytics.gold.reporting_dim_product_gold_SCDTUPE1 
like pricing_analytics.gold.reporting_dim_product_gold

In [0]:
create or replace table pricing_analytics.silver.reporting_dim_product_stage_1 as 
select distinct PRODUCT_NAME, PRODUCTGROUP_NAME 
from pricing_analytics.silver.daily_pricing_silver
where lakehouse_updated_date >
(select nvl(max(PROCESSED_TABLE_DATE_TIME), "1900-01-01") from pricing_analytics.processrunlogs.deltalakehouse_process_runs
where PROCESS_NAME="reportingDimensionTableLoadSCDType1" and  PROCESS_STATUS= "Completed")

In [0]:
select * from pricing_analytics.silver.reporting_dim_product_stage_1

In [0]:
create or replace table pricing_analytics.silver.reporting_dim_product_stage_2 as 
select silverDim.PRODUCT_NAME,
silverDim.PRODUCTGROUP_NAME,
goldDim.PRODUCT_NAME as GOLD_PRODUCT_NAME,
(case when goldDim.PRODUCT_NAME is null
 then row_number() over (order by silverDim.PRODUCT_NAME, silverDim.PRODUCTGROUP_NAME) 
 else 
 goldDim.PRODUCT_ID end 
) as PRODUCT_ID,
current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date 
from pricing_analytics.silver.reporting_dim_product_stage_1 silverDim
left outer join pricing_analytics.gold.reporting_dim_product_gold_SCDTUPE1 goldDim
on silverDim.PRODUCT_NAME = goldDim.PRODUCT_NAME
where isnull(goldDim.PRODUCT_NAME) or silverdim.PRODUCTGROUP_NAME != goldDim.PRODUCTGROUP_NAME

In [0]:
select * from pricing_analytics.silver.reporting_dim_product_stage_2

In [0]:
create or replace table  pricing_analytics.silver.reporting_dim_product_stage_3 as 
select silverDim.PRODUCT_NAME,
silverDim.PRODUCTGROUP_NAME,
case when PRODUCT_NAME is null then (silverDim.PRODUCT_ID + PreviousMax) else silverDim.PRODUCT_ID end as PRODUCT_ID,
current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date 
from pricing_analytics.silver.reporting_dim_product_stage_2 as silverDim
cross join (select nvl(max(goldDim.PRODUCT_ID),0) as PreviousMax  from  pricing_analytics.gold.reporting_dim_product_gold as goldDim)

In [0]:
select * from pricing_analytics.silver.reporting_dim_product_stage_3

In [0]:
Merge into pricing_analytics.gold.reporting_dim_product_gold_scdtupe1 as goldDim 
using  pricing_analytics.silver.reporting_dim_product_stage_3 as silverDim 
on goldDim.PRODUCT_ID = silverDim.PRODUCT_ID
when matched then 
update set goldDim.PRODUCTGROUP_NAME = silverDim.PRODUCTGROUP_NAME
 ,goldDim.lakehouse_updated_date = current_timestamp()
when not matched then 
insert *

In [0]:
select * from pricing_analytics.gold.reporting_dim_product_gold_scdtupe1 where PRODUCT_NAME ='Onion'

In [0]:
insert into pricing_analytics.processrunlogs.deltalakehouse_process_runs(PROCESS_NAME,PROCESS_STATUS,PROCESSED_TABLE_DATE_TIME)
select "reportingDimensionTableLoadSCDType1", "Completed", max(lakehouse_updated_date) from pricing_analytics.silver.daily_pricing_silver